In [ ]:
%%capture
# installation
!git clone https://github.com/speechbrain/speechbrain/
%cd /content/speechbrain/
!pip install -r requirements.txt
!pip install -e .

In [ ]:
%%capture
# For pip installation
!pip install speechbrain
!mkdir "/content/results"
!mkdir "/content/common_voice"

Load pretrained speech-brain model. 
Speech brain has 68 different models to choose from listed on hugging face ~ https://huggingface.co/speechbrain 

In [12]:
import torchaudio
from speechbrain.pretrained import EncoderClassifier
# Model is downloaded from the speechbrain HuggingFace repo (There could be others that work better).

classifier = EncoderClassifier.from_hparams(
  source="speechbrain/spkrec-ecapa-voxceleb",
  savedir="/content/results",
)

(Optional) Load the english common voice dataset from Hugging Face.

In [14]:
import torch
import numpy as np
import os

# TODO: Add in the mfccs and transcripts for the finetuning. 
class AudioDataset(torch.utils.data.Dataset):
    def __init__(self, root):
        self.waveforms = []
        self.accents = []

        mp3_file_names = os.listdir(root)
        print("The length of mp3 file names is {}".format(len(mp3_file_names)))
        mp3_file_names.sort()
        for i in range(len(mp3_file_names)):
            waveform, sample_rate = torchaudio.load(root + mp3_file_names[i])
            print("The shape of waveforms is {}".format(waveform.shape))
            self.waveforms.append(waveform)
            self.accents.append("TODO?")
        
        assert(len(self.waveforms) == len(self.accents))
        self.length = len(self.waveforms)

    def __len__(self):
        return self.length


    def __getitem__(self, ind):
        waveform = self.waveforms[ind]
        return waveform, self.accents[ind]

    def collate_fn(self, batch):
      batch_waveforms = []
      batch_lengths = []
      for i in range(len(batch)):
          batch_waveforms.append(batch[i]) 
          batch_lengths.append(len(batch[i]))

      return torch.tensor(batch_waveforms), torch.tensor(batch_lengths)

common_voice_dataset = AudioDataset("/content/common_voice/")

The length of mp3 file names is 0


Dataloader

In [15]:
common_voice_data_loader = torch.utils.data.DataLoader(
    dataset     = common_voice_dataset, 
    num_workers = 2, 
    batch_size  = 1, 
    pin_memory  = True, 
    shuffle     = False
)

Compute embeddings for the dataset. 

In [17]:
embeddings = []
for i, (waveform_batch, accent_batch) in enumerate(common_voice_data_loader):
  
  # The arguments to encode batch should be wav, wav_lens
  # wav is shape [batch_size, time, num_channels] or just [batch_size, time]
  # wav_lens is a torch.Tensor of shhape [batch] the longest recording in the 
  # batch has length 1.0. All other waveforms have a relative length to the 
  # max length. i.e. waveform.shape[1]/len(max_length)
  #print(type(waveform_batch))
  #print(waveform_batch.shape)
  #waveform_batch.permute(0,2,1)
  waveform_lens = torch.tensor([1.0])
  embedding = classifier.encode_batch(waveform_batch, waveform_lens)
  waveform = waveform_batch.squeeze()
  embedding = classifier.encode_batch(waveform)
  embeddings.append(embedding)

print("The length of embeddings is {}".format(len(embeddings)))
print("The shape of embedding 0 is {}".format(embeddings[0].shape))


The length of embeddings is 0


IndexError: ignored

TODO: Create a new tensor which contains the original recording concatenated with the embedding.

Finetune the Whisper ASR model with the embedding + original common voice waveform. I am following the tutorial from ~ https://huggingface.co/blog/fine-tune-whisper. 

Load Pretrained Whisper Model for ASR

In [ ]:
!pip install transformers
from transformers import WhisperFeatureExtractor
from transformers import WhisperTokenizer

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="English", task="transcribe")

ModuleNotFoundError: ignored

Apply the data preparation function over the training data. 

Define a data collator

Perform inference using the GMU Speech Accent Archive.